# unmapped unique gene counts

Note the of htseqCount.out has the following additional information. TODO: add them to output
```
ENSG00000288698.1	0
ENSG00000288699.1	0
__no_feature	1541926
__ambiguous	12650
__too_low_aQual	0
__not_aligned	0
__alignment_not_unique	159190
```

In [1]:
import numpy as np
import pandas as pd

kl = "/private/groups/kimlab"
krasDir= kl + "/kras.ipsc/data"
salmonIdx = "gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx"

In [2]:
findPath= krasDir + "/{bulk.data,exo.data}"
htSeqCountFiles = !find $findPath -type f -name htseqCount.out | grep $salmonIdx
print( htSeqCountFiles[0:3] )

assert len(htSeqCountFiles) == 18

['/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.1/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out', '/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.2/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out', '/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.3/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out']


In [3]:
retDF = pd.DataFrame( {"htSeqCountFile": htSeqCountFiles} )
pd.set_option('display.max_colwidth', None)
retDF.head()

,htSeqCountFile
0,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.1/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out
1,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.2/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out
2,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.3/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out
3,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/kras.1/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out
4,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/kras.2/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out


In [4]:
# trim root off of paths
pathRoot = "/private/groups/kimlab/kras.ipsc/data/"
retDF[ "sampleId" ] = retDF.loc[:,"htSeqCountFile"].str.replace( pathRoot, "", regex=True)
# print()
# print( retDF['sampleId'].head() )
# print( "retDF['sampleId'].shape: {}".format(retDF["sampleId"].shape) )


# remove any counts that do not match our ref
retDF = retDF[ retDF.loc[:,"htSeqCountFile"].str.contains( salmonIdx ) ]
# print( "retDF['sampleId'].shape: {}".format(retDF["sampleId"].shape) )


startPattern = "/htseqCount.out"
endPattern = "htseqCount.out"
pattern = "{}.*{}".format( startPattern, endPattern )          
retDF[ "sampleId" ] = retDF[ "sampleId" ].str.replace( pattern, "", regex=True )

print( retDF["sampleId"].head() )

0    bulk.data/day.5/ctrl.1
1    bulk.data/day.5/ctrl.2
2    bulk.data/day.5/ctrl.3
3    bulk.data/day.5/kras.1
4    bulk.data/day.5/kras.2
Name: sampleId, dtype: object


In [5]:
retDF.head()

,htSeqCountFile,sampleId
0,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.1/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,bulk.data/day.5/ctrl.1
1,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.2/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,bulk.data/day.5/ctrl.2
2,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.3/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,bulk.data/day.5/ctrl.3
3,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/kras.1/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,bulk.data/day.5/kras.1
4,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/kras.2/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,bulk.data/day.5/kras.2


In [6]:
def cleanDF( df ):
    # htseq-count has upto 5 rows at the bottom of file containg
    # stats about unmapped reads. see Top cell for more details
    selectRows = df['gene'].str.contains( 'ENSG', regex=True ) 
    retGenesDF = df[selectRows]
    retStatsDF = df[ np.logical_not(selectRows)]

    # htseq-count reports counts not tpm
    selectNonZeroRows = (retGenesDF[ "count" ] != 0) 
    retGenesDF = retGenesDF.loc[ selectNonZeroRows, ["gene", "count"] ]
    
#     print("AEDWIP sum(selectNonZeroRows):{}".format(sum(selectNonZeroRows)))
#     print("AEDWIP retGenesDF['count'].count():{}".format(retGenesDF['count'].count()))
#     print("AEDWIP retGenesDF.shape:{}".format(retGenesDF.shape))
    return (retGenesDF, retStatsDF)


def countUniqueGenes( htSeqCountFiles ):
    # pre allocate list to store number of unique genes 
    # in each sample
    n = len( htSeqCountFiles )
    numUniqeGenesList = [0] * n
    filePathList = [""] * n # use for debug

    for i in range(len(htSeqCountFiles)) :
#         if i > 0:
#             print("AEDWIP debug")
#             break

        countPath = htSeqCountFiles[i]
        print("countPath: {}".format(countPath))
        df = pd.read_csv( countPath, sep="\t", header=0, names=["gene", "count"])
#         print("\n****** before clean")
#         print("df.shape:{}".format(df.shape))
        retGenesDF, retStatsDF = cleanDF( df )
    #     print("\n******** afer clean")
    #     print("df.shape:{}".format(df.shape))
    #     print( "df['geneId'].count(): {}".format(df["geneId"].count()) )  
    #     print( "len( pd.unique(df['geneId'] )): {}".format( len(pd.unique( df['geneId']) )) )

        # future proof
        # htseq returns gene counts. so no need to group
        # in the future It may be possible to get to produce more interesting counts
        groupDFs = retGenesDF.groupby(["gene"])
        groupDict = groupDFs.groups
        groupKeys = [ k for k in groupDict.keys() ]
        #print(groupKeys[0:4])
        numUniqueGenes = len(groupKeys)
        print("len(groupKeys): {}".format(len(groupKeys)) )

        numUniqeGenesList[i] = numUniqueGenes
        filePathList[i] = countPath
        
    return (numUniqeGenesList, filePathList)

In [7]:
numUniqeGenesList, filePathList = countUniqueGenes( htSeqCountFiles )
# add the results list as col to pd
retDF['numUnMappedUniqGenes'] = numUniqeGenesList
#retDF['debug'] = filePathList
#retDF

countPath: /private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.1/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out
len(groupKeys): 9577
countPath: /private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.2/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out
len(groupKeys): 9925
countPath: /private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.3/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out
len(groupKeys): 9911
countPath: /private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/kras.1/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out
len(groupKeys): 9466
countPath: /private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/kras.2/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out
len(groupKeys): 8232
countPath: /private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/

# write output

In [8]:
#outputPath = "../data/uniqueUnMappedCounts.tsv"
pwd= !pwd
outputPath = pwd[0] + "/../data/uniqueUnMappedCounts.tsv"
print("outputPath:\n{}\n".format(outputPath))

# reorder output
retDF.loc[:, ["sampleId", "numUnMappedUniqGenes", "htSeqCountFile"] ].to_csv(outputPath, sep="\t", index=False)

outputPath:
/private/home/aedavids/unmappedReadsAnalysis/jupyterNotebooks/../data/uniqueUnMappedCounts.tsv



In [9]:
retDF.head()

,htSeqCountFile,sampleId,numUnMappedUniqGenes
0,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.1/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,bulk.data/day.5/ctrl.1,9577
1,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.2/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,bulk.data/day.5/ctrl.2,9925
2,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.3/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,bulk.data/day.5/ctrl.3,9911
3,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/kras.1/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,bulk.data/day.5/kras.1,9466
4,/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/kras.2/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,bulk.data/day.5/kras.2,8232


In [10]:
retDF.tail()

,htSeqCountFile,sampleId,numUnMappedUniqGenes
13,/private/groups/kimlab/kras.ipsc/data/exo.data/gen1c.day.5.exo.input.data/ctrl.2/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,exo.data/gen1c.day.5.exo.input.data/ctrl.2,9549
14,/private/groups/kimlab/kras.ipsc/data/exo.data/gen1c.day.5.exo.input.data/ctrl.3/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,exo.data/gen1c.day.5.exo.input.data/ctrl.3,10734
15,/private/groups/kimlab/kras.ipsc/data/exo.data/gen1c.day.5.exo.input.data/kras.1/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,exo.data/gen1c.day.5.exo.input.data/kras.1,13986
16,/private/groups/kimlab/kras.ipsc/data/exo.data/gen1c.day.5.exo.input.data/kras.2/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,exo.data/gen1c.day.5.exo.input.data/kras.2,14092
17,/private/groups/kimlab/kras.ipsc/data/exo.data/gen1c.day.5.exo.input.data/kras.3/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out,exo.data/gen1c.day.5.exo.input.data/kras.3,15008


# test

In [11]:
f = "/private/groups/kimlab/kras.ipsc/data/bulk.data/day.5/ctrl.1/htseqCount.out/STAR.gencode.v35.ucsc.rmsk.salmon.v1.3.0.sidx/unmapped/unmapped/htseqCount.out"
df = pd.read_csv( f, sep="\t", header=0, names=["gene", "count"])
df.head()
print(df.shape)
retGenesDF, retStatsDF = cleanDF( df )
print(retGenesDF.shape)
print(retStatsDF)

groupDFs = retGenesDF.groupby(["gene"])
groupDict = groupDFs.groups
groupKeys = [ k for k in groupDict.keys() ]
#print(groupKeys[0:4])
numUniqueGenes = len(groupKeys)
numUniqueGenes

(60655, 2)
(9577, 2)
                         gene    count
60650            __no_feature  1541926
60651             __ambiguous    12650
60652         __too_low_aQual        0
60653           __not_aligned        0
60654  __alignment_not_unique   159190


9577